# Requirements

In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/pavelpopov/courses/digital image processing/project'

In [2]:
import math, re
import tensorflow as tf
import numpy as np
import glob

2023-11-20 21:24:43.738844: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Read tfrec

In [9]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [512, 512, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

In [10]:
train_paths = glob.glob("/Users/pavelpopov/courses/digital image processing/project/assets/data/original_data/train/*")
val_paths = glob.glob("/Users/pavelpopov/courses/digital image processing/project/assets/data/original_data/train/*")
test_paths = glob.glob("/Users/pavelpopov/courses/digital image processing/project/assets/data/original_data/train/*")

In [13]:
train_dataset = tf.data.TFRecordDataset(train_paths).map(read_labeled_tfrecord)

In [33]:
save_path = "/Users/pavelpopov/courses/digital image processing/project/assets/data/np_data"
data = []
labels = []
i = 0
j = 0
for example in train_dataset:
    data.append(example[0].numpy())
    labels.append(example[1].numpy())
    i += 1
    if i >= 1000:
        data = np.stack(data)
        labels = np.stack(labels)
        print(data.shape)
        np.savez(f"{save_path}/train_data_{j}.npz", data=data, labels=labels)

        data = []
        labels = []
        j += 1
        i = 0

data = np.stack(data)
labels = np.stack(labels)
print(data.shape)
# np.savez(f"{save_path}/train_data.npz", data=data, labels=labels)
np.savez(f"{save_path}/train_data_{j}.npz", data=data, labels=labels)

(1000, 512, 512, 3)
(1000, 512, 512, 3)
(1000, 512, 512, 3)
(1000, 512, 512, 3)
(1000, 512, 512, 3)
(1000, 512, 512, 3)
(1000, 512, 512, 3)
(1000, 512, 512, 3)
(1000, 512, 512, 3)
(1000, 512, 512, 3)
(1000, 512, 512, 3)
(1000, 512, 512, 3)
(753, 512, 512, 3)


In [ ]:
save_path = "/Users/pavelpopov/courses/digital image processing/project/assets/data/np_data"
data = []
labels = []
i = 0
j = 0
for example in train_dataset:
    data.append(example[0].numpy())
    labels.append(example[1].numpy())
    i += 1
    if i >= 1000:
        data = np.stack(data)
        labels = np.stack(labels)
        print(data.shape)
        np.savez(f"{save_path}/train_data_{j}.npz", data=data, labels=labels)

        data = []
        labels = []
        j += 1
        i = 0

data = np.stack(data)
labels = np.stack(labels)
print(data.shape)
# np.savez(f"{save_path}/train_data.npz", data=data, labels=labels)
np.savez(f"{save_path}/train_data_{j}.npz", data=data, labels=labels)

In [40]:
save_path = "/Users/pavelpopov/courses/digital image processing/project/assets/data/np_data"
data = []
labels = []
i = 0
j = 0
for example in train_dataset:
    # data.append(example[0].numpy())
    labels.append(example[1].numpy())
    # i += 1
    # if i >= 1000:
    #     data = np.stack(data)
    #     labels = np.stack(labels)
    #     print(data.shape)
    #     np.savez(f"{save_path}/train_data_{j}.npz", data=data, labels=labels)

    #     data = []
    #     labels = []
    #     j += 1
    #     i = 0

# data = np.stack(data)
# labels = np.stack(labels)
# print(data.shape)
# # np.savez(f"{save_path}/train_data.npz", data=data, labels=labels)
# np.savez(f"{save_path}/train_data_{j}.npz", data=data, labels=labels)

In [55]:
labels_np = np.stack(labels)
labels_per_class = {class_idx: np.where(labels_np == class_idx)[0] for class_idx in np.unique(labels_np)}

In [ ]:
save_path = "/Users/pavelpopov/courses/digital image processing/project/assets/data/np_data/data_per_class"
for class_idx, positions in labels_per_class:
    data = np.load(f"/Users/pavelpopov/courses/digital image processing/project/assets/data/np_data/train_data_{class_idx}.npz")